In [1]:
import os
import numpy as np
from prostate_dataset import ProstateDataset
dirname = os.path.join(os.getcwd(), 'cedars-224')
data = ProstateDataset(dirname)

In [2]:
_, mask, class_ids, scores = data.read_mask_rcnn_output(12)

In [3]:
def prediction_2_sementic(mask, class_ids, scores):
    """ convert maskrcnn prediction to 1 single sementic mask
    """
    mask_map = {} # create a probability dict for each class
    for index, label in enumerate(class_ids):
        mask_map[str(label)] = np.maximum(mask_map[str(label)], scores[index] * mask[:, :, index])\
        if str(label) in mask_map.keys() else scores[index] * mask[:, :, index]
    
    ## convert to h * w * num_classes probability map
    h, w, d = mask.shape
    for i in range(4):
        try:
            sementic_mask = np.concatenate((sementic_mask, np.expand_dims(mask_map[str(i)], axis = -1)),axis = -1)\
            if (i != 0) else np.expand_dims(mask_map[str(i)], axis = -1)
        except KeyError:
            sementic_mask = np.concatenate((np.zeros((h, w, 1)), mask_map[str(i)]),axis = -1)\
            if (i != 0) else np.zeros((h, w, 1))
            
    sementic_mask_res = np.argmax(sementic_mask, axis = -1)
    return sementic_mask_res


In [4]:
mask = prediction_2_sementic(mask, class_ids, scores)
mask_decode = data.convert_mat_annotations_to_png(mask)
data.show_ann_png(mask_decode)

In [6]:
from sklearn.metrics import confusion_matrix
temp = confusion_matrix(np.ravel(mask), np.ravel(mask))

In [9]:
print(temp + temp)
print(temp)

[[1254292       0       0       0]
 [      0   71752       0       0]
 [      0       0   11678       0]
 [      0       0       0  759430]]
[[627146      0      0      0]
 [     0  35876      0      0]
 [     0      0   5839      0]
 [     0      0      0 379715]]


In [10]:
i = 0
PA = temp[i, i] / (sum(temp[i, :]) + sum(temp[:, i]) - temp[i, i])
print(PA)

1.0


In [11]:
def compute_mIOU(c_matrix):
    """ compute the mIOU based on the confusion matrix
    """
    num_class, _ = c_matrix.shape
    IOU = []
    for i in range(num_class):
        p = c_matrix[i, i] / (sum(c_matrix[i, :]) + sum(c_matrix[:, i]) - c_matrix[i, i])
        IOU.append(p)
    return np.mean(IOU)

In [12]:
print(compute_mIOU(temp))

1.0


In [13]:
print(mask.shape)

(1024, 1024)


In [18]:
orig_mask = data.read_original_ann(12)
h, w = mask.shape

In [19]:
orig_mask_crop = orig_mask[0:h, 0:w]
temp = confusion_matrix(np.ravel(orig_mask_crop), np.ravel(mask))
print(compute_mIOU(temp))

0.118820004702
